<a href="https://colab.research.google.com/github/bongkyunSON/Deep_Leaning/blob/main/DL_221202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import os
# os.cpu_count()
!nvidia-smi

Fri Dec  2 05:34:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **Deep Learning Track. Mini PJT - CIFAR100 classification**


저희는 수업시간에 CIFAR10을 분류하는 문제를 풀어보았습니다.

이 때 까지 배운 모든 방법을 사용하여, CIFAR100 데이터셋을 100가지로 분류하는 문제를 풀어보세요.
프로젝트를 수행할 때, 다음 조건들을 만족하여야 합니다.

----

1.  MLP, (Vanilla)CNN, ResNet50 중에 2개의 모델을 골라서 구현합니다.


2. 각 모델을 design하여 (hyper-parameter를 세팅) 구현합니다.


3. Validation Accuracy 기준 0.4를 넘기는 모델과 학습 세팅을 찾습니다.

----

> 모델을 구현하고 성능을 올리기 위한 팁들을 몇 가지 소개합니다.

- learning rate가 optimization에 가장 큰 영향을 끼칩니다. 다양한 숫자를 테스트해보세요.


- 처음에 학습이 잘되는 세팅을 찾기 위해서는 10 epoch 정도로 초반 학습 성능이 잘나오는지 확인하는게 중요합니다. (val acc도 포함)


- 초반 학습 성능이 잘 나왔다면, epochs를 늘려서 학습이 잘되는지 확인합니다.


- epoch를 늘려도 한계에 도달했다면, overfitting이 되었는지 아닌지 확인합니다.


- overfitting이 되었다면, 여러 regularization 기법들이 도움이 될 겁니다. e.g. tensorflow.keras.layers.Dropout


- batch size도 바꿔보세요. batch size가 full-batch에 가까울 수록, 속도는 느려지지만 optimizer의 방향은 최적의 값을 향합니다.


- MLP나 CNN을 구현하고 있다면, 모델의 크기를 바꿔보세요. 파라미터가 많을 수록, 학습할 수 있는 정보가 많아집니다.

In [2]:
# Import Libraries
import numpy as np
import tensorflow as tf
import random
import os

# cifar10과 keras layer들 불러오기
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Sequential

In [3]:
## mini batch 데이터의 변경을 막기 위한 random seed 고정
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(0xC0FFEE)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [4]:
# Load Datasets
(X_train, y_train), (X_test, y_test) = cifar100.load_data()
print(X_train.shape) # 4차원 Tensor
print(X_test.shape)

### Preprocessing

1. Normalization (v)


2. Standardization

In [ ]:
preprocessing = "standard" # "standard"
debug_mode = False

In [ ]:
# Normalization (Minmax scaling)
if preprocessing == "normalize":
    X_train = X_train.astype(np.float32) / 255.0
    X_test = X_test.astype(np.float32) / 255.

In [6]:
# Standarization
if preprocessing == "standard":
    if debug_mode:
        print("-- Before --")
        print("Training data\n", X_train[0])

    img_mean = np.mean(X_train, axis=0)
    img_std = np.std(X_train, axis=0)

    if debug_mode:
        print(img_mean.shape)
        print(img_std.shape)

        print("mean : ", np.mean(X_train)) # through data
        print("std : ", np.std(X_train))

    # Scaling
    X_train = X_train.astype(np.float32) - img_mean
    X_train = X_train.astype(np.float32) / img_std

    if debug_mode:
        print("\n\n-- After --")
        #print("Training data\n", X_train[0])
        print("mean : ", np.mean(X_train))
        print("std : ", np.std(X_train))
    
    # Scaling
    X_test = X_test.astype(np.float32) - img_mean
    X_test = X_test.astype(np.float32) / img_std

In [ ]:
X_train[0]

### Training with Vanilla CNN

**Model Architecture**

> Conv - Relu - Conv - Relu - Pool - Conv - Relu - Conv - Relu - Pool - FC - Softmax

In [73]:
# 위의 Vanilla CNN 구조를 직접 구현해봅니다.

model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(filters=8, kernel_size=3, strides=1, padding='same', activation='relu'),
    Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'),
    MaxPool2D(pool_size=2),
    Conv2D(24, 3, 1, padding='same', activation='relu'),
    Conv2D(48, 3, 1, padding='same', activation='relu'),
    MaxPool2D(2),
    Flatten(),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(100, activation='softmax') # softmax
])

model.summary()

157/157 [==============================] - 1s 4ms/step - loss: 2.8908 - accuracy: 0.3376
Loss : 2.8908, Accuracy : 0.3376


In [ ]:
# training setup
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

# val_loss 기준 체크포인터도 생성합니다.
cnn_filename = "CNN_best_model.ckpt"
checkpoint = ModelCheckpoint(cnn_filename, 
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)


## config variables
batch_size = 50000
lr = 1e-3
epochs = 30

# learning rate를 업데이트 해보려고 합니다.
def scheduler(epoch, learning_rate):
    if epoch < 25:
        return learning_rate
    elif epoch < 50:
        return lr/10
    elif epoch < 75:
        return lr/50
    else:
        return lr/100    

# Set optimizer, loss function, metrics, callback function
#optimizer = SGD(learning_rate=lr, momentum=0.9) # Momentum optimizer 
optimizer = Adam(learning_rate=lr)
loss_fn = 'sparse_categorical_crossentropy' # multi-class classification
metrics = ['accuracy']
lr_callback = LearningRateScheduler(scheduler)

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)

In [ ]:
# model training
# 한번만 직접 해봅시다!

history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=[lr_callback, checkpoint],
                    validation_data=(X_test, y_test))

In [ ]:
# evaluate
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print("Loss : %.4f, Accuracy : %.4f" % (loss, acc))

In [ ]:
## loss visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.subplot(1, 2,1)
plt.plot(history.history['loss'],'b-', label = "training")
plt.plot(history.history['val_loss'], 'r:', label = "validation")
plt.title("model - loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.title("model - accuracy")

plt.plot(history.history['accuracy'], 'b-', label = "training")
plt.plot(history.history['val_accuracy'], 'r:', label = "validation")

plt.legend()
plt.tight_layout()
plt.show()

### 이번엔 ResNet50를 사용해봅시다.

- ResNet50을 사용하기 위해서는 keras에 구현되어 있는 ResNet50 구조를 가져옵니다.

- pretrained를 가져오는 경우도 있지만, 지금은 ImageNet 세팅이 아니므로 구조만 가져옵니다.(input이 완전히 다름)

- Reference : https://keras.io/api/applications/resnet/#resnet50-function

In [ ]:
# load ResNet50
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import ResNet152

resnet = ResNet50(
    include_top=True,        # classification : True,  embedding : False
    weights=None,             # ImageNet pretrained : True, else : None
    input_shape=(32, 32, 3),  # if not pretrained, you must specify.
    pooling='max',
    classes=100,               # number of classes
    classifier_activation='softmax'    # output function
)

In [ ]:
# see ResNet50 model
#resnet.summary()

In [ ]:
### resnet training strategy
# val_loss 기준 체크포인터도 생성합니다.
resnet_filename = "resnet_best_model.ckpt"
checkpoint = ModelCheckpoint(resnet_filename, 
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)


## config variables
batch_size = 64
lr = 5e-3
epochs = 30

# learning rate를 업데이트 해보려고 합니다
def scheduler(epoch, learning_rate):
    if epoch < 5:
        return learning_rate
    elif epoch < 10:
        return lr/10
    elif epoch < 15:
        return lr/50
    else:
        return lr/100
    
#optimizer = SGD(learning_rate=lr, momentum=0.9)
optimizer = Adam(learning_rate=lr)
loss_fn = 'sparse_categorical_crossentropy'
metrics = ['accuracy']

resnet.compile(optimizer=optimizer,
               loss=loss_fn,
               metrics=metrics)

In [ ]:
# training ResNet50
resnet_history = resnet.fit(X_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(X_test, y_test),
                            callbacks=[lr_callback, checkpoint],
                            verbose=1)

In [ ]:
# evaluate
loss, acc = resnet.evaluate(X_test, y_test, batch_size=batch_size)
print("Loss : %.4f, Accuracy : %.4f" % (loss, acc))

In [ ]:
## loss visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.subplot(1, 2,1)
plt.plot(resnet_history.history['loss'],'b-', label = "training")
plt.plot(resnet_history.history['val_loss'], 'r:', label = "validation")
plt.title("model - loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.title("model - accuracy")

plt.plot(resnet_history.history['accuracy'], 'b-', label = "training")
plt.plot(resnet_history.history['val_accuracy'], 'r:', label = "validation")

plt.legend()
plt.tight_layout()
plt.show()